# 데이터 받아오기

In [1]:
from modules.scouter_handler import ScouterHandler
from modules.doc_graph_analyzer import DocGraphAnalyzer
import numpy as np

In [2]:
keyword = "대통령"

min_rel_score = 2.0
max_length = 3000

In [3]:
scouter = ScouterHandler(size=500)

In [4]:
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'node', 'edge', 'extContent'])
doc_info_list = scouter.search(query_body, data_type="graphs", trim_lower=True)

Query : 대통령
Scroll idx : 1 (500 docs)
Scroll idx : 2 (500 docs)
Scroll idx : 3 (500 docs)
Scroll idx : 4 (500 docs)
Scroll idx : 5 (500 docs)
Scroll idx : 6 (500 docs)
Scroll idx : 7 (500 docs)
Scroll idx : 8 (475 docs)
Trim lower 70% of docs (1192 docs are cut)
Total retrieved Doc # :  2782



In [5]:
doc_info_list = [d for d in doc_info_list if d['rel_score'] > min_rel_score and len(d['extContent']) < max_length]
num_doc = len(doc_info_list)
num_doc

2761

In [6]:
doc_temp_id_list = {doc_info['news_id']:i for i, doc_info in enumerate(doc_info_list)}

# 문서 클러스터링

In [48]:
## modified by sspark ##
doc_graph_anal = DocGraphAnalyzer(keyword, df_bound=(0.01, 0.80))
doc_graph_anal.extract_subtopic_info(doc_info_list, top_doc_num=5, top_keyword_num=5, hierarchy=True, cutting_level=2)
## modified by sspark ##

Doc #  : 2761
Node # : 1669
Edge # : 330990

[0] Community 0/0 :   76 Keyword, (검찰/조사/소환/수사/이명박), len_topdocs : 128, topdoc 5 : [41941, 103736, 104651, 54836, 88720]
[1] Community 0/1 :  104 Keyword, (혐의/구속/특수/사건/서울중앙), len_topdocs : 179, topdoc 5 : [55358, 55010, 62101, 20741, 44677]
[2] Community 0/2 :   77 Keyword, (의원/자금/의혹/회장/불법), len_topdocs : 15, topdoc 5 : [38032, 104586, 105136, 36953, 56363]
[3] Community 0/3 :  123 Keyword, (다스/관련/소유/자신/사실), len_topdocs : 167, topdoc 5 : [95109, 6673, 27075, 26549, 14858]
[4] Community 1/0 :  106 Keyword, (실장/국가/안보/특사/상황), len_topdocs : 53, topdoc 5 : [26199, 24229, 58903, 27043, 37765]
[6] Community 1/2 :  120 Keyword, (미국/트럼프/한국/시간/도널드), len_topdocs : 154, topdoc 5 : [91641, 23372, 39969, 40233, 17979]
[9] Community 1/5 :  174 Keyword, (대화/관계/남북/정상/회담), len_topdocs : 902, topdoc 5 : [37631, 27414, 55801, 95797, 24059]
[11] Community 2/2 :   54 Keyword, (대표/청와대/이날/문재/참석), len_topdocs : 1, topdoc 5 : [94986]
[12] Community 3/0 :   79 Keyword

# 결과 분석

In [46]:
chosen_docidx = 84627

In [47]:
print(doc_temp_id_list[chosen_docidx], doc_info_list[doc_temp_id_list[chosen_docidx]]['extContent'])

1799 울산시교육청이 민원서비스 평가에서 대통령 표창을 받는 것은 처음이며, 2016년 시도교육청 가운데 3위(S등급)로 선정된 바 있다. 행정안전부와 국민권익위원회가 주관하는 이 평가는 2013년부터 시작됐다. 시도교육청(17개), 중앙행정기관(42개), 광역지자체(17개), 기초지자체(226개) 등 총 302개 기관이 대상인데, 이번에 기초지자체까지 확대됐다. 평가 내용은 민원만족도 향상, 민원답변 충실도, 민원행정 전략과 수행체계, 민원제도 운용, 고객만족도 향상과 민원서비스 개선 정도를 평가한다. 울산교육청은 이번 평가에서 학생인권 민원, 자사고의 일반고 전환 민원, 공동주택 학교신설 민원 등 다양하고 열악한 여건에서 우수한 성과를 달성해 좋은 점수를 얻은 것으로 분석했다. 이는 전 직원이 교육수요자인 학생, 학부모, 교직원을 위하는 마음으로 적극적인 업무자세와 관심으로 최상의 민원서비스에 나선 결과라고 교육청은 설명했다. 울산교육청은 매월 기관장이 민원처리 현황을 파악하고, 다양한 교육수요자 민원만족도 향상을 위해 신속하게 답변하고, 전화 방문하며 민원인과 소통했다. 이 밖에 민원사전 예약제 홈페이지 기능 운영, 민원처리 우수공무원 선진지 견학과 포상 등 직원 사기진작 등을 위해 노력했다. 교육청 관계자는  최고의 만족, 우선 공감, 수요자 중심의 교육민원 서비스 제공을 위해 민원인 입장에서 불편함을 개선하고 민원인의 이해와 공감을 얻는 서비스를 하도록 노력할 것 이라고 밝혔다.


In [32]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis=1).reshape(x.shape[0], 1))
    return e_x / e_x.sum(axis=1).reshape(x.shape[0], 1)
    
a = softmax(doc_graph_anal.relevance_obj.np_docsNsubtopic_rel)
a = doc_graph_anal.relevance_obj.np_docsNsubtopic_rel

In [36]:
a[doc_temp_id_list[chosen_docidx]] / sum(a[doc_temp_id_list[chosen_docidx]])

array([0.32181846, 0.        , 0.0687883 , 0.02607333, 0.        ,
       0.00178828, 0.0265625 , 0.02903192, 0.02319298, 0.00729561,
       0.        , 0.        , 0.00112241, 0.0584558 , 0.02102312,
       0.        , 0.        , 0.00895469, 0.10336268, 0.02076594,
       0.        , 0.16906906, 0.        , 0.00376755, 0.00764999,
       0.03568645, 0.        , 0.        , 0.        , 0.        ,
       0.04980739, 0.        , 0.        , 0.0079352 , 0.        ,
       0.00378759, 0.00406075, 0.        , 0.        ])

# Keywords

In [15]:
comm_index = 11
blank_size = 8
for i in range(0, len(doc_graph_anal.docgraph_obj.list_subgraph[comm_index].vs['label']), blank_size):
    for j in doc_graph_anal.docgraph_obj.list_subgraph[comm_index].vs['label'][i:i+blank_size]:
        print(doc_graph_anal.raw_doc_obj.idx2keyword[j], end="\t")
    print()

대표	청와대	이날	문재	이후	참석	비서	회동	
여야	수석	오늘	처음	차례	비판	미투	취임	
오찬	모습	임종석	정무	농담	논평	거부	민정	
모두	북콘서트	당사	불참	간담	한병	영수	여의도	
결심	마찬가지	기도	본관	정동기	세상	고통	상춘	
등장	부탁	장하성	강훈	환담	장제원	무사	눈길	
거절	진짜	김현	반문	화제	신년	


# Scouter를 사용한 결과 분석

In [ ]:
top_doc_id_query_list = [scouter.make_doc_id_query_body(top_doc_ids, filters=["news_id", "newsTitle", "extContent"])
                         for top_doc_ids in doc_graph_anal.top_doc_id_list]
top_doc_info_list = [scouter.search(doc_id_query, data_type='newspaper', trim_lower=False, silence=True)
                     for doc_id_query in top_doc_id_query_list]